In [27]:
%pip install nlpaug
%pip install nltk


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
import os

import pandas as pd

In [29]:
original_dataset = os.path.join(os.path.abspath('outputs'), 'response_quality_dataset.csv')


df_r_quality = pd.read_csv(original_dataset)

df_r_quality

,responses,quality
0,I you know you don't have to sign up if you do...,0
1,because it seems like I you know you've expres...,0
2,so why do you eat too much I mean how is that ...,0
3,so you like it you just like eating food so yo...,0
4,yeah yeah,0
...,...,...
4582,on a scale of one to ten where one is not at a...,1
4583,why'd you say eight or nine rather than six or...,1
4584,all right what do you think is a realistic goa...,1
4585,I prefer just make sure your goal is kind of t...,1


In [30]:
import nltk
import pandas as pd
import nlpaug.augmenter.word as naw
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jonaslindstroem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:
'''
Note: This cell is responsible for augmentating through the use of synonyms this is done by appending
      all records of the dataset and substituting all the synonyms.

'''

from nltk.corpus import wordnet as wn

# Initialize the augmenter with 'wordnet' as a string
aug = naw.SynonymAug(aug_src="wordnet", aug_max=4)
augmented_df = pd.DataFrame(columns=['responses', 'quality'])

# Iterate over the rows in the original DataFrame
for index, row in df_r_quality.iterrows():
    aug_res = aug.augment(row['responses'])
    # Append the augmented data to augmented_df
    temp_df = pd.DataFrame({'responses': aug_res, 'quality': row['quality']})
    augmented_df = pd.concat([augmented_df, temp_df], ignore_index=True)

# Combine the original and augmented data
combined_df = pd.concat([df_r_quality, augmented_df], ignore_index=True)

combined_df

,responses,quality
0,I you know you don't have to sign up if you do...,0
1,because it seems like I you know you've expres...,0
2,so why do you eat too much I mean how is that ...,0
3,so you like it you just like eating food so yo...,0
4,yeah yeah,0
...,...,...
9169,on a scale of measurement of 1 to ten where on...,1
9170,why ' d you say eight or nine sooner than sest...,1
9171,all correct what do you guess is a realistic g...,1
9172,I favour just make sure your goal is kind of t...,1


In [32]:
'''
Note: This cell is responsible for data augmentation though the use of letter substituion
      to avoid degrading the quality of the dataset, only a portion of the dataset is
      substituted.

'''

import nlpaug.augmenter.word as naw


# Take a random selection
df_rows = combined_df.size // 2
random_selection = combined_df.sample(n=int(df_rows * 0.4))

# Create the augment object
aug = naw.RandomWordAug(action="swap")

# Iterate over the rows in the original DataFrame
for index, row in combined_df.iterrows():
    aug_res = aug.augment(row['responses'])
    # Append the augmented data to augmented_df
    temp_df = pd.DataFrame({'responses': aug_res, 'quality': row['quality']})
    augmented_df = pd.concat([augmented_df, temp_df], ignore_index=True)

# Combine the original and augmented data
combined_df = pd.concat([combined_df, augmented_df], ignore_index=True)

combined_df


,responses,quality
0,I you know you don't have to sign up if you do...,0
1,because it seems like I you know you've expres...,0
2,so why do you eat too much I mean how is that ...,0
3,so you like it you just like eating food so yo...,0
4,yeah yeah,0
...,...,...
22930,on scale of a measurement of 1 to where ten on...,1
22931,' why d you say or nine eight sooner than sest...,1
22932,all correct what guess do is you a realistic g...,1
22933,I favour just make sure your goal is kind of t...,1


In [33]:
output_file = os.path.join(os.path.abspath('outputs'), 'final_augmented_dataset.csv')
# Write DataFrame to CSV file
combined_df.to_csv(output_file, index=False)

print(f"DataFrame is written to {output_file}")

DataFrame is written to /Users/jonaslindstroem/Programs/therapist/therapist_model/models/response_classifier/quality_response_dataset/outputs/final_augmented_dataset.csv
